Import Libraries

In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts

Read in and Format Data

In [117]:
names = ['Age', 'Temperature of Water', 'Length of Fish']
df_x = pd.read_csv('x06Simple.csv', usecols=range(1,4), names=names, header=0, engine='python')
df_x = df_x.sample(frac=1, random_state=0)
df_y = pd.DataFrame(data=df_x[df_x.columns[-1]])
df_x.drop(df_x.columns[[-1,]], axis=1, inplace=True)

Split x and y Data into Test and Training Sets

In [118]:
df_x_train, df_x_test, df_y_train, df_y_test = tts(df_x, df_y, test_size=0.33)

Calculate Mean and Standard Deviation to Standardize x Data

In [119]:
mean = df_x_train.mean(axis=0)
std = df_x_train.std(axis=0)
df_x_s_train = df_x_train.subtract(mean).divide(std)
df_x_s_test = df_x_test.subtract(mean).divide(std)
x = np.hstack([np.ones((df_x_s_train.shape[0], 1)), df_x_s_train.values])
y = df_y_train.values

Define RMSE Function

In [120]:
def RMSE(thetas, x, y):
    x = np.hstack([np.ones((x.shape[0], 1)), x.subtract(mean).divide(std)])
    diff = y.subtract(x @ thetas)
    return np.sqrt(diff.pow(2, axis=0).sum(axis=0).values/float(len(y)))

Find Distances, Weights, and Thetas. Use Them to Find the Average RMSE

In [125]:
rmse = 0
for x_test in df_x_s_test.values:
    num_rows = df_x_s_train.shape[0]
    W = np.mat(np.eye(num_rows))
    k = 1
    for i in range(num_rows):
        x_train = df_x_s_train.values[i]
        d = np.abs(x_test[0] - x_train[0]) + np.abs(x_test[1] - x_train[1])
        W[i, i] = np.exp(-d / (k ** 2))
    thetas = np.linalg.inv(x.T @ W @ x) @ (x.T @ W @ y)
    rmse += RMSE(thetas, df_x_test, df_y_test)
rmse /= df_x_test.shape[0]
print(rmse)

[515.26881075]


As you can see, the average RMSE is 515.26881075